# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaのコンペティションにおいて、どちらのチャットボット応答がユーザーに好まれるかを予測するためのモデルをトレーニングするプロセスを示しています。具体的には、2つの大規模言語モデル（LLM）から生成された応答を比較し、選好を予測するための一連の手法を適用しています。

主な取り組み内容としては、以下の手順が含まれています：

1. **ライブラリのインポート**: `transformers`, `torch`, `peft`, `datasets`などのライブラリを用いて、モデルのトレーニング、推論、およびデータ処理を行います。

2. **データの準備**: CSVファイルからトレーニングデータとテストデータを読み込み、JSON形式のテキストデータを処理してプロンプトと応答を結合する関数を作成します。この段階で、欠損値も処理されています。

3. **トークナイザーの準備**: 指定したトークナイザーを用いてテキストをトークナイズし、入力IDとアテンションマスクを生成します。

4. **モデル設定とロード**: GPUを2つ使ってモデルを設定し、LoRA（Low-Rank Adaptation）技術を利用してメモリ効率を最大化しながら8ビットの計算でモデルをロードします。その後、トレーニング済みの重みを読み込みます。

5. **推論の実行**: スレッドプールを使用し、バッチごとにデータを処理しながらモデルの推論を行います。結果として、Aモデルが勝つ確率、Bモデルが勝つ確率、引き分けの確率を計算します。

6. **提出ファイルの生成**: 最終的な予測結果を新しいデータフレームにまとめ、"submission.csv"という名前でCSVファイルとして保存します。

このNotebookでは、特に`transformers`ライブラリの`AutoModelForSequenceClassification`やLoRA設定、また`PeftModel`を活用してモデルのメモリ効率とトレーニングの性能を向上させる努力がなされています。ユーザーの好みを効果的に予測するために、複数のモデルの出力を比較するアプローチが取られています。

---


# 用語概説 
以下は、提供されたJupyter Notebook内で機械学習や深層学習の初心者がつまずきそうな専門用語の解説です。基本的な知識を持つ方を対象にしているため、より詳細で特有の知識に焦点を当てています。

1. **PEFT (Parameter-Efficient Fine-Tuning)**:
   - モデルのパラメータを全て再訓練するのではなく、一部のパラメータのみを調整してファインチューニングを行う手法です。計算資源を削減しつつ、特定のタスクに適応させるのが目的です。このNotebookでは、LoRA設定を用いてPEFTを活用しています。

2. **LoRA (Low-Rank Adaptation)**:
   - モデルのパラメータ行列を低次元に近似することによって、効率的にパラメータ数を削減しつつ、モデルを特定のタスクに適応させる方法です。具体的には、更新対象の層を少数のランクで表現し、計算の負荷を下げることができます。

3. **Causal Language Model (CLM)**:
   - 言語モデルの一種で、テキストを生成する際に、過去の単語情報のみを考慮に入れて次の単語を予測します。計算された出力は、前の文脈に基づくもので、時系列データや文章生成において主に使用されます。

4. **Attention Mask**:
   - テキストを処理する際に、どのトークンが重要か、または無視されるべきかを示すためのマスクです。特にパディングされたトークンが計算に影響を及ぼさないようにするために使用され、機械学習モデルがトレーニングされる際のコンテキスト情報を保持する役割を果たします。

5. **Softmax Function**:
   - モデルの出力に対して確率分布を生成するための関数です。出力の値を正規化し、各クラスの予測確率を計算するために使用されます。このNotebookでは、モデルの結果に対してソフトマックス関数を適用することで、勝者確率を得ています。

6. **Automatic Mixed Precision (AMP)**:
   - 学習中に使用するデータ型を16ビットと32ビットの浮動小数点で自動的に切り替える技術です。これにより、計算速度を向上させ、GPUメモリの使用効率を高めることができます。特に大規模なモデルの場合、重要な最適化手法となります。

7. **Batch Size**:
   - モデルのパラメータを更新する際に、同時に処理するデータサンプルの数です。バッチサイズを調整することで、トレーニングの安定性や効率を管理することができます。

8. **ThreadPoolExecutor**:
   - Pythonのconcurrent.futuresライブラリに含まれるクラスで、スレッドによる並列処理を簡単に実現するためのものです。大規模なデータ処理や推論を効率よく行うために使用され、リソースを最大限に活用します。

9. **BitsAndBytes**:
   - モデルを8ビット量子化するための設定を行うライブラリです。これにより、モデルのメモリ使用量を削減し、計算の効率を向上させます。特に大規模なモデルを扱う際に重要です。

10. **Data Collator**:
    - データのバッチを形成するためのロジックを持つクラスまたは関数です。特に異なる長さの入力データを指定された形式でまとめるために使用され、パディングなどの処理を行います。特定のデータ形式に対応したデータコラレーターの使用がNotebook内で示されています。

これらの用語は、初心者の方には馴染みがない場合が多いので、特に留意して解説しました。Notebookの内容を理解する際に役立つ情報となるでしょう。

---


# 注記
- [トレーニングスクリプト](https://www.kaggle.com/code/shelterw/training-llama3-8b-4-bit-qlora-sft)

# インポート


In [ ]:
!pip install transformers -U --no-index --find-links /kaggle/input/lmsys-transformers/lmsys_transformers

In [ ]:
!pip install peft accelerate bitsandbytes -U --no-index --find-links /kaggle/input/lmsys-wheel-files

In [ ]:
import json
import time
import sklearn
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.cuda.amp import autocast
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
from threading import Thread
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig, DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType 
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import BitsAndBytesConfig, LlamaForCausalLM, LlamaModel, LlamaPreTrainedModel
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from transformers import set_seed

# 高度なメモリ効率を利用するための設定
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)
# 利用可能なGPUが2つあることを確認します
assert torch.cuda.device_count() == 2, "申し訳ありませんが、マルチGPUが必要です！"
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# モデルの設定
MODEL_NAME = '/kaggle/input/meta-llama-3-8b/LLM-Research/Meta-Llama-3-8B'  # モデル名を指定
WEIGHTS_PATH = '/kaggle/input/llama31-sample5500-cls/llama31-sample5500-cls/checkpoint-550'  # 重みのパス
TOKENIZER_PATH = '/kaggle/input/llama31-sample5500-cls/llama31-sample5500-cls/tokenizer'  # トークナイザーのパス
MAX_LENGTH = 2400  # 最大シーケンス長
BATCH_SIZE = 4  # バッチサイズ設定
DEVICE = torch.device("cuda")  # CUDAデバイスを指定

# データの準備 


In [ ]:
def process_text(text: str) -> list:
    # JSON形式のテキストを解析し、欠損値を'none'で置き換えます
    x = json.loads(text)
    x = ['none' if pd.isna(i) else i for i in x]
    return x

def merge_text(x):
    # プロンプトとレスポンスを結合します
    prompt = x['prompt']
    response_a = x['response_a']
    response_b = x['response_b']
    res = ''
    for i in range(len(prompt)):
        if i == len(prompt) - 1:
            res += f'<prompt>: {prompt[i]}' + f'\n\n<response_a>: {response_a[i]}' + f'\n\n<response_b>: {response_b[i]}'
        else:
            res += f'<prompt>: {prompt[i]}' + f'\n\n<response_a>: {response_a[i]}' + f'\n\n<response_b>: {response_b[i]}' + '\n\n'
    return res

In [ ]:
# トレーニングデータとテストデータを読み込みます
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# テストデータの行数が3のときは、トレーニングデータの最初の100行を使用します
if test.shape[0] == 3:
    test = train.head(100)

# 各列を処理します
test['prompt'] = test['prompt'].map(lambda x: process_text(x))
test['response_a'] = test['response_a'].map(lambda x: process_text(x))
test['response_b'] = test['response_b'].map(lambda x: process_text(x))

# プロンプトとレスポンスを結合して新しいテキスト列を作ります
test['text'] = test.apply(lambda x: merge_text(x), axis=1)
# テストデータの最初の行を表示します
test.head()

# トークナイズ


In [ ]:
def tokenize(tokenizer, x):
    # テキストをトークナイズして、入力IDとアテンションマスクを生成します
    tokenized = tokenizer(x, max_length=MAX_LENGTH, truncation=True)
    return tokenized['input_ids'], tokenized['attention_mask']

In [ ]:
# トークナイザーをロードします
llama31_tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

# 空のデータフレームを作成
llama31_data = pd.DataFrame()
llama31_data["id"] = test["id"]
# トークナイズした結果をデータフレームに保存します
llama31_data["input_ids"], llama31_data["attention_mask"] = tokenize(llama31_tokenizer, list(test['text']))
# 入力IDの長さを計算します
llama31_data["length"] = llama31_data["input_ids"].apply(len)

# モデルをロード 
それぞれのGPUに1つのモデルをロードします。  


In [ ]:
# 8ビットの計算に関する設定を定義します
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=False,
)

device_0 = torch.device('cuda:0')  # 最初のGPUを指定
llama31_model_0 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # 出力ラベルの数
    torch_dtype=torch.float16,  # 16ビット浮動小数点でモデルをロード
    quantization_config=bnb_config,
    use_cache=False,  # キャッシュを無効に
    device_map=device_0  # デバイスの設定
)
# パディングトークンIDを設定します
llama31_model_0.config.pad_token_id = llama31_tokenizer.pad_token_id

device_1 = torch.device('cuda:1')  # 2つ目のGPUを指定
llama31_model_1 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    use_cache=False,
    device_map=device_1
)
# パディングトークンIDを設定します
llama31_model_1.config.pad_token_id = llama31_tokenizer.pad_token_id

# LoRA設定の作成
lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    # 自己注意のターゲットモジュールのみ
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    lora_dropout=0,  # ドロップアウト率を0に設定
    bias='none',  # バイアスを無効に
    task_type=TaskType.SEQ_CLS,  # タスクタイプをシーケンス分類に設定
)

# モデルのトレーニング準備についてのコードはコメントアウトされています
# llama31_model_0 = prepare_model_for_kbit_training(llama31_model_0)
# llama31_model_0 = get_peft_model(llama31_model_0, lora_config)
# llama31_model_0.print_trainable_parameters()

# llama31_model_1 = prepare_model_for_kbit_training(llama31_model_1)
# llama31_model_1 = get_peft_model(llama31_model_1, lora_config)
# llama31_model_1.print_trainable_parameters()

# 重みのロード 


In [ ]:
# PEFTを取得します
llama31_model_0 = PeftModel.from_pretrained(llama31_model_0, model_id=WEIGHTS_PATH).to(device_0)
llama31_model_0.eval()  # 評価モードに設定

llama31_model_1 = PeftModel.from_pretrained(llama31_model_1, model_id=WEIGHTS_PATH).to(device_1)
llama31_model_1.eval();  # 評価モードに設定

# 推論



In [ ]:
@torch.no_grad()  # 勾配を計算しないようにします
@torch.cuda.amp.autocast()  # 自動混合精度を使用します
def inference(df, model, tokenizer, device, batch_size=BATCH_SIZE, max_length=MAX_LENGTH):
    a_win, b_win, tie = [], [], []  # 各モデルの勝者を格納するリスト
    
    # バッチごとにデータを処理
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]  # 現在のバッチを取得
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストに変換
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクをリストに変換
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",  # 最も長いシーケンスに合わせてパディング
            pad_to_multiple_of=None,
            return_tensors="pt",  # PyTorchテンソルとして返す
        )
        outputs = model(**inputs.to(device))  # モデルへの入力
        proba = outputs.logits.softmax(-1).cpu()  # ソフトマックス関数を適用して確率を計算
        
        # 各モデルの勝者の確率を収集
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    df["winner_model_a"] = a_win  # 勝者モデルAの確率をデータフレームに追加
    df["winner_model_b"] = b_win  # 勝者モデルBの確率をデータフレームに追加
    df["winner_tie"] = tie  # 引き分けの確率をデータフレームに追加
    
    return df  # データフレームを返す

In [ ]:
st = time.time()  # 処理開始の時間を記録

# テストデータを長さでソート
llama31_data = llama31_data.sort_values("length", ascending=False).reset_index(drop=True)
sub_1 = llama31_data.iloc[0::2].copy()  # 偶数の行を取得
sub_2 = llama31_data.iloc[1::2].copy()  # 奇数の行を取得

# スレッドプールを使用して推論を並列処理
with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (llama31_model_0, llama31_model_1), (llama31_tokenizer, llama31_tokenizer), (device_0, device_1))

# 結果を結合
result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 各モデルの勝者確率を取得

print(f"経過時間: {time.time() - st}")  # 処理にかかった時間を表示

In [ ]:
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝者確率を設定
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝者確率を設定
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 引き分け確率を設定
# 提出用データフレームを作成
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
# CSVファイルとして保存
submission_df.to_csv('submission.csv', index=False)
# 提出データフレームを表示
display(submission_df)